### Gillespie algorithm for simulating kinesin walking on microtubules in vitro
MK Iwanski 2020-04-07

In [1]:
# imports
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# function for debugging
def pause():
    programPause = raw_input("Press <ENTER> to continue.")

In [3]:
# parameters
mt_length = 100
nstep = 10**4

#rates of reactions
k_fwd = 0.01 #steps per sec
k_bck = 0 #steps per sec
k_on = 0.002 #per dimer per sec
k_off = 0.3 #per sec

#update vectors for each reaction
nu_fwd = -1
nu_bck = -1
nu_on = 1
nu_off = -1

#update affinities of mt sites
alpha_max = 0.0001 #change in k_on at site of motor
#TO DO: assume this drops exponentially with number of sites away from motor and with time

In [11]:
def gillespie(X, t, m): #update time and state vectors (mt and affinity at site m)
   
    #generate random numbers
    rt = np.random.random()
    rr = np.random.random()
    
    #time to next reaction
    tau = np.log(1/rt)/k_tot
    
    #determine which reaction
    j = 0
    test = k[0]/k_tot
    while test < rr and j < len(k)-1:
        t += k[j]/k_tot
        j += 1

    t += tau #update time
    X[m] += nu[j] #update state
    if m < mt_length-1 and X[m+1] == 0 and j == 0:
        X[m+1] -= nu[j] #update state of +1 site for forward step
    if m >  0 and X[m-1] == 0 and j == 1:
         X[m-1] -= nu[j] #update state of -1 site for back step
    
    return X, t

In [12]:
def affinity_update(aff, X): #update affinity based on motor positions
    aff[X==1] = alpha_max #update affinity
    motors = np.where(X==1)
    for motor_loc in motors:
        for j in range(0,mt_length):
            while motor_loc+j<mt_length-1:
                aff[motor_loc+j] = alpha_max*np.exp(-j)
            while motor_loc-j>=0:
                aff[motor_loc-j] = alpha_max*np.exp(-j)
    #add to update of surrounding sites
    aff[aff<0] = k_on
    return aff

In [13]:
#initialize
mt = np.zeros((nstep,mt_length), dtype=np.int)
affinity = np.zeros((nstep,mt_length))
affinity[:] = k_on
t = np.zeros(nstep)

for i in range(0,nstep-1):
    #choose MT lattice site
    m = np.random.randint(0,mt_length-1)
    #check current possible reactions
    if mt[i,m] == 1: #site is occupied
        k = [k_fwd, k_bck, 0, k_off]
        k_tot = np.sum(k)
        nu = [nu_fwd, nu_bck, 0, nu_off]
    else: #site is empty
        k = [0, 0, k_on, k_off]
        k_tot = np.sum(k)
        nu = [0, 0, nu_on, 0]
        
    #run Gillespie
    mt[i+1,:],t[i+1] = gillespie(mt[i,:],t[i],m)
    affinity[i+1,:] = affinity_update(affinity[i,:],mt[i+1,:])
#pause() 

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  


In [10]:
display(mt[-3,])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])